## Exploración y Validación de Datos - Enfermedad Cardíaca

### By:
Laura Granda

### Date:
2026-02-22

### Description:

Notebook para explorar, validar y preparar datos cargados desde la etapa anterior. Se cargarán los datos desde el parquet intermedio, se validarán valores nulos (eliminando filas si alguna columna >10% nulos), se analizarán tipos de datos de cada característica, se convertirán a tipos apropiados, y se guardarán los datos limpios en formato parquet para la siguiente etapa.

## 📚 Import libraries

In [39]:
# Base libraries for data science
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

# Display options for better visualization
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

## 💾 Load data

In [40]:
# Load the heart disease dataset from intermediate stage
data_path = "/home/lauragranda01/corazon/data/02_intermediate/corazon_raw.parquet"
df = pd.read_parquet(data_path, engine="pyarrow")

print(f"Dataset shape: {df.shape}")
print(f"Dataset loaded successfully from: {data_path}")

Dataset shape: (3030, 14)
Dataset loaded successfully from: /home/lauragranda01/corazon/data/02_intermediate/corazon_raw.parquet


## 👀 Basic Information

In [41]:
# Display first few rows
print("=" * 80)
print("FIRST 10 ROWS OF THE DATASET")
print("=" * 80)
print(df.head(10))

FIRST 10 ROWS OF THE DATASET
  age     sex    chest_pain rest_bp chol  fbs                       rest_ecg  \
0  63    Male       typical     145  233  1.0  left ventricular hypertrophy    
1  67    Male  asymptomatic     160  286  0.0  left ventricular hypertrophy    
2  67    Male  asymptomatic     120  229  0.0  left ventricular hypertrophy    
3  37    Male    nonanginal     130  250  0.0                         normal   
4  41  Female    nontypical     130  204  0.0  left ventricular hypertrophy    
5  56    Male    nontypical     120  236  0.0                         normal   
6  62  Female  asymptomatic     140  268  0.0  left ventricular hypertrophy    
7  57  Female  asymptomatic     120  354  0.0                         normal   
8  63    Male  asymptomatic     130  254  0.0  left ventricular hypertrophy    
9  53    Male  asymptomatic     140  203  1.0  left ventricular hypertrophy    

  max_hr exang old_peak slope   ca        thal disease  
0    150     0      2.3     3  0.

In [42]:
# Data info: types and missing values
print("\n" + "=" * 80)
print("DATA TYPES AND MISSING VALUES")
print("=" * 80)
data_info = pd.DataFrame(
    {
        "Column": df.columns,
        "Data Type": df.dtypes,
        "Missing Count": df.isnull().sum(),
        "Missing %": (df.isnull().sum() / len(df) * 100).round(2),
        "Unique Values": [df[col].nunique() for col in df.columns],
    }
)
print(data_info.to_string(index=False))


DATA TYPES AND MISSING VALUES
    Column Data Type  Missing Count  Missing %  Unique Values
       age    object             30       0.99             43
       sex    object             61       2.01              5
chest_pain    object             83       2.74              7
   rest_bp    object             81       2.67             52
      chol    object             85       2.81            154
       fbs   float64             97       3.20              2
  rest_ecg    object            193       6.37              8
    max_hr    object            171       5.64             92
     exang    object            151       4.98              4
  old_peak    object            150       4.95             42
     slope    object            151       4.98              4
        ca    object            162       5.35              5
      thal    object            126       4.16              7
   disease    object            106       3.50              7


In [43]:
# Descriptive statistics
print("\n" + "=" * 80)
print("DESCRIPTIVE STATISTICS")
print("=" * 80)
print(df.describe().to_string())


DESCRIPTIVE STATISTICS
               fbs
count  2933.000000
mean      0.148312
std       0.355470
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000


## 🔍 Null Values Validation

In [ ]:
# Step 2: Clean numeric columns (remove spaces, line breaks, validate)
print("\n2️⃣ Cleaning NUMERIC columns (whitespace, line breaks)...")
numeric_cols = ["age", "rest_bp", "chol", "max_hr", "old_peak"]
numeric_cols = [col for col in numeric_cols if col in df.columns]

for col in numeric_cols:
    # Count issues before cleaning
    issues_found = 0

    # Remove leading/trailing whitespace and line breaks
    if df[col].dtype == "object":
        df[col] = df[col].astype(str).str.strip().str.replace(r"\s+", "", regex=True)
        issues_found = (df[col] == "").sum() + (df[col].isnull()).sum()

    # Replace empty strings with NaN
    df.loc[df[col] == "", col] = np.nan

    # Try to convert to numeric and catch conversion errors
    try:
        # Convert and coerce errors to NaN
        df[col] = pd.to_numeric(df[col], errors="coerce")
        invalid_count = df[col].isnull().sum() - df[col].isna().sum()

        if invalid_count > 0:
            print(
                f"  ⚠️ {col}: Cleaned whitespace/line breaks, {invalid_count} invalid values → NaN"
            )
        else:
            print(f"  ✅ {col}: All values are valid numeric")
    except Exception as e:
        print(f"  ⚠️ {col}: Error during conversion - {e!s}")

# Step 2b: Clean categorical/ordinal columns (strip whitespace)
print("\n2️⃣b Cleaning CATEGORICAL/ORDINAL columns (whitespace)...")
categorical_ordinal_cols = ["sex", "chest_pain", "thal", "rest_ecg", "slope", "ca"]
categorical_ordinal_cols = [col for col in categorical_ordinal_cols if col in df.columns]

for col in categorical_ordinal_cols:
    if df[col].dtype == "object":
        # Strip whitespace from strings
        df[col] = df[col].astype(str).str.strip()
        # Try to convert strings that look like numbers to numeric
        df[col] = pd.to_numeric(df[col], errors="ignore")
        print(f"  ✅ {col}: Whitespace removed, cleaned")
    else:
        print(f"  ✅ {col}: Already cleaned")


2️⃣ Cleaning NUMERIC columns (whitespace, line breaks)...
  ✅ age: All values are valid numeric
  ✅ rest_bp: All values are valid numeric
  ✅ chol: All values are valid numeric
  ✅ max_hr: All values are valid numeric
  ✅ old_peak: All values are valid numeric

2️⃣b Cleaning CATEGORICAL/ORDINAL columns (whitespace)...
  ✅ sex: Whitespace removed, cleaned
  ✅ chest_pain: Whitespace removed, cleaned
  ✅ thal: Whitespace removed, cleaned
  ✅ rest_ecg: Whitespace removed, cleaned
  ✅ slope: Whitespace removed, cleaned
  ✅ ca: Whitespace removed, cleaned


## 📋 Data Types Analysis

In [45]:
# Detailed analysis of each feature
print("=" * 80)
print("DATA TYPES AND FEATURE CLASSIFICATION")
print("=" * 80)

feature_analysis = {}

# Define all possible features with their metadata
all_features = {
    "age": {
        "classification": "Numerical - Continuous",
        "description": "Patient age in years",
        "target_type": "int",
    },
    "sex": {
        "classification": "Categorical - Nominal",
        "description": "Patient gender (Male/Female)",
        "target_type": "category",
    },
    "chest_pain": {
        "classification": "Categorical - Nominal",
        "description": "Type of chest pain (typical, atypical, asymptomatic, nontypical)",
        "target_type": "category",
    },
    "rest_bp": {
        "classification": "Numerical - Continuous",
        "description": "Resting blood pressure (mm Hg)",
        "target_type": "int",
    },
    "chol": {
        "classification": "Numerical - Continuous",
        "description": "Serum cholesterol (mg/dl)",
        "target_type": "int",
    },
    "fbs": {
        "classification": "Binary",
        "description": "Fasting blood sugar > 120 mg/dl (True/False)",
        "target_type": "bool",
    },
    "rest_ecg": {
        "classification": "Categorical - Ordinal",
        "description": "Resting ECG results (0,1,2 = normal, ST-T abnormality, LV hypertrophy)",
        "target_type": "int",
    },
    "max_hr": {
        "classification": "Numerical - Continuous",
        "description": "Maximum heart rate achieved (bpm)",
        "target_type": "int",
    },
    "exang": {
        "classification": "Binary",
        "description": "Exercise induced angina (True/False)",
        "target_type": "bool",
    },
    "old_peak": {
        "classification": "Numerical - Continuous",
        "description": "ST depression induced by exercise relative to rest",
        "target_type": "float",
    },
    "slope": {
        "classification": "Categorical - Ordinal",
        "description": "Slope of peak exercise ST segment (0,1,2 = upsloping, flat, downsloping)",
        "target_type": "int",
    },
    "ca": {
        "classification": "Categorical - Ordinal",
        "description": "Number of major vessels (0-3) colored by fluoroscopy",
        "target_type": "int",
    },
    "thal": {
        "classification": "Categorical - Nominal",
        "description": "Thalemia/myocardial perfusion defect (normal, fixed, reversible)",
        "target_type": "category",
    },
    "disease": {
        "classification": "Binary - TARGET",
        "description": "Presence of heart disease (True/False)",
        "target_type": "bool",
    },
}

# Build analysis only for columns that exist in current dataframe
for col in df.columns:
    if col in all_features:
        metadata = all_features[col]
        unique_vals = df[col].nunique()

        # Get unique values representation based on dtype
        if df[col].dtype == "bool":
            unique_repr = str(df[col].unique())
        elif df[col].dtype in ["int64", "float64"]:
            unique_repr = f"{df[col].min()}-{df[col].max()}"
        else:
            unique_repr = str(df[col].unique())[:100]

        feature_analysis[col] = {
            "current_type": str(df[col].dtype),
            "classification": metadata["classification"],
            "description": metadata["description"],
            "target_type": metadata["target_type"],
            "unique_values": unique_repr,
        }

# Display analysis table
df_analysis = pd.DataFrame(feature_analysis).T
print(df_analysis.to_string())

DATA TYPES AND FEATURE CLASSIFICATION
           current_type          classification                                                               description target_type                                                                                          unique_values
age             float64  Numerical - Continuous                                                      Patient age in years         int                                                                                              29.0-77.0
sex              object   Categorical - Nominal                                              Patient gender (Male/Female)    category                                                             ['Male' 'Female' '2345' '45' '765' 'None']
chest_pain       object   Categorical - Nominal          Type of chest pain (typical, atypical, asymptomatic, nontypical)    category                     ['typical' 'asymptomatic' 'nonanginal' 'nontypical' '2345' '2435' '3456'\n 'None']
rest_bp       

## 🔄 Data Type Conversion

In [ ]:
# Convert data types with proper classifications
print("=" * 80)
print("CONVERTING DATA TYPES")
print("=" * 80)

# Convert NOMINAL categorical to category
categorical_nominal_cols = ["sex", "chest_pain", "thal"]
categorical_nominal_cols = [col for col in categorical_nominal_cols if col in df.columns]

if categorical_nominal_cols:
    print("\nConverting NOMINAL categorical columns to 'category' type...")
    for col in categorical_nominal_cols:
        df[col] = df[col].astype("category")
        print(f"  ✅ {col}: {df[col].dtype}")

# Convert BINARY columns to bool
binary_cols = ["fbs", "exang", "disease"]
binary_cols = [col for col in binary_cols if col in df.columns]

if binary_cols:
    print("\nConverting BINARY columns to 'bool'...")
    for col in binary_cols:
        # First ensure numeric, then convert to bool
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col] = df[col].astype("bool")
        print(f"  ✅ {col}: {df[col].dtype}")

# Convert ORDINAL categorical to category with defined ordering
ordinal_cols = ["rest_ecg", "slope", "ca"]
ordinal_cols = [col for col in ordinal_cols if col in df.columns]

# Define ordinal ordering for each categorical column
ordinal_ordering = {
    "rest_ecg": ["normal", "ST-T wave abnormality", "left ventricular hypertrophy"],
    "slope": ["1", "2", "3"],  # upsloping, flat, downsloping
    "ca": ["0.0", "1.0", "2.0", "3.0"],  # number of vessels (0-3)
}

if ordinal_cols:
    print("\nConverting ORDINAL categorical columns to 'category' with ordering...")
    for col in ordinal_cols:
        # Get unique non-null values to preserve actual order in data
        unique_vals = df[col].dropna().unique().tolist()
        # Define ordered categorical with natural order
        if col in ordinal_ordering:
            # Filter ordering to only include values present in data
            order = [v for v in ordinal_ordering[col] if v in unique_vals]
            df[col] = pd.Categorical(df[col], categories=order, ordered=True)
        else:
            df[col] = df[col].astype("category")
        print(
            f"  ✅ {col}: {df[col].dtype} | Order: {list(df[col].cat.categories) if hasattr(df[col], 'cat') else 'N/A'}"
        )

# Convert CONTINUOUS numerical columns to int
continuous_cols = ["age", "rest_bp", "chol", "max_hr"]
continuous_cols = [col for col in continuous_cols if col in df.columns]

print("\nVerifying CONTINUOUS numerical columns...")
for col in continuous_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
    if df[col].dtype != "int64":
        df[col] = df[col].astype("Int64")  # Nullable integer type
    print(f"  ✅ {col}: {df[col].dtype}")

# Convert FLOAT columns
if "old_peak" in df.columns:
    df["old_peak"] = pd.to_numeric(df["old_peak"], errors="coerce")
    df["old_peak"] = df["old_peak"].astype("float")
    print(f"  ✅ old_peak: {df['old_peak'].dtype}")

CONVERTING DATA TYPES

Converting NOMINAL categorical columns to 'category' type...
  ✅ sex: category
  ✅ chest_pain: category
  ✅ thal: category

Converting BINARY columns to 'bool'...
  ✅ fbs: bool
  ✅ exang: bool
  ✅ disease: bool

Converting ORDINAL categorical columns to 'category' with ordering...
  ✅ rest_ecg: category | Order: ['normal', 'ST-T wave abnormality', 'left ventricular hypertrophy']
  ✅ slope: category | Order: ['1', '2', '3']
  ✅ ca: category | Order: ['0.0', '1.0', '2.0', '3.0']

Verifying CONTINUOUS numerical columns...
  ✅ age: Int64
  ✅ rest_bp: Int64
  ✅ chol: Int64
  ✅ max_hr: Int64
  ✅ old_peak: float64


In [47]:
# Verify all conversions
print("\n" + "=" * 80)
print("FINAL DATA TYPES")
print("=" * 80)
print(df.dtypes)


FINAL DATA TYPES
age              Int64
sex           category
chest_pain    category
rest_bp          Int64
chol             Int64
fbs               bool
rest_ecg      category
max_hr           Int64
exang             bool
old_peak       float64
slope         category
ca            category
thal          category
disease           bool
dtype: object


## 📊 Analysis of Results and Conclusions

In [ ]:
# Summary of data preparation
print("=" * 80)
print("DATA PREPARATION SUMMARY")
print("=" * 80)

print(f"\n✅ Final dataset shape: {df.shape}")

# Categorize column types
categorical_cols_final = [col for col in df.columns if df[col].dtype == "category"]
bool_cols_final = [col for col in df.columns if df[col].dtype == "bool"]
float_cols_final = [col for col in df.columns if df[col].dtype == "float64"]
int_cols_final = [col for col in df.columns if df[col].dtype == "Int64"]

# Separate categorical columns into nominal and ordinal
ordinal_categorical_cols = [
    col for col in categorical_cols_final if col in ["rest_ecg", "slope", "ca"]
]
nominal_categorical_cols = [
    col for col in categorical_cols_final if col not in ordinal_categorical_cols
]

print("\n📊 Data type distribution:")
print(f"  - Categorical (Nominal): {nominal_categorical_cols}")
print(f"  - Categorical (Ordinal): {ordinal_categorical_cols}")
if ordinal_categorical_cols:
    print("\n    📋 Ordinal categories (with order):")
    for col in ordinal_categorical_cols:
        if hasattr(df[col], "cat") and df[col].cat.ordered:
            order = list(df[col].cat.categories)
            print(f"      • {col}: {order}")
print(f"  - Binary: {bool_cols_final}")
print(f"  - Numerical (Continuous/Int): {int_cols_final}")
print(f"  - Numerical (Float): {float_cols_final}")

# Show target variable distribution if it exists
if "disease" in df.columns:
    print("\n🎯 Target variable (disease) distribution:")
    print(df["disease"].value_counts().sort_index())

print("\n✨ Data is ready for feature engineering stage")

DATA PREPARATION SUMMARY

✅ Final dataset shape: (3030, 14)

📊 Data type distribution:
  - Categorical (Nominal): ['sex', 'chest_pain', 'thal']
  - Categorical (Ordinal): ['rest_ecg', 'slope', 'ca']

    📋 Ordinal categories (with order):
      • rest_ecg: ['normal', 'ST-T wave abnormality', 'left ventricular hypertrophy']
      • slope: ['1', '2', '3']
      • ca: ['0.0', '1.0', '2.0', '3.0']
  - Binary: ['fbs', 'exang', 'disease']
  - Numerical (Continuous/Int): ['age', 'rest_bp', 'chol', 'max_hr']
  - Numerical (Float): ['old_peak']

🎯 Target variable (disease) distribution:
disease
False    1576
True     1454
Name: count, dtype: int64

✨ Data is ready for feature engineering stage


## 💾 Save Cleaned Data

In [52]:
# Export validated and cleaned data to parquet format
output_path = "/home/lauragranda01/corazon/data/03_primary/corazon_primary.parquet"

df.to_parquet(output_path, engine="pyarrow", index=False)

print(f"✅ Data saved successfully to: {output_path}")
print(f"   Shape: {df.shape}")
print(f"   Columns: {list(df.columns)}")

✅ Data saved successfully to: /home/lauragranda01/corazon/data/03_primary/corazon_primary.parquet
   Shape: (3030, 14)
   Columns: ['age', 'sex', 'chest_pain', 'rest_bp', 'chol', 'fbs', 'rest_ecg', 'max_hr', 'exang', 'old_peak', 'slope', 'ca', 'thal', 'disease']
